In [1]:
import numpy as np
import math
from src.texas_holdem_simp import Texas_Holdem_Simp

env = Texas_Holdem_Simp()
env.plot(env.enum(('J♥', 'J♠', 'Q♥', 'Q♠', 3,0,0)))

----(1)-----
|P2(5): J♠ |
|(2): Q♥-**|
|P1(3): J♥ |
------------


In [2]:
def uct(max_iter=1000, max_d=10, gamma=.9, beta=1):
    Q = np.zeros((env.S.size, env.A.size))
    n = np.zeros((env.S.size, env.A.size))

    def search(s, d):
        if s in env.G:
            return 0
        if d == max_d:
            return np.max([env.R(s,a,s_) for s_ in env.S for a in env.A if env.T(s,a,s_)>0])
        
        expl = lambda a: ((np.log(np.sum(n[s]))/n[s,a])**.5) if n[s,a]>0 else 0
        # a = np.argmax([Q[s,a] + beta*expl(a) for a in env.A])
        v = np.array([Q[s,a] + beta*expl(a) for a in env.A])
        a = np.random.choice(np.flatnonzero(v == v.max()))
        if abs(np.sum([env.T(s,a,s_) for s_ in env.S]) - 1)>0.01:
            print(s, a)
        s_ = np.random.choice(env.S, p=[env.T(s,a,s_) for s_ in env.S])
        r = env.R(s,a,s_)
        q = r + gamma*search(s_, d+1)
        Q[s,a] = (n[s,a]*Q[s,a]+q) / (n[s,a]+1)
        n[s,a] += 1
        return q

    for i in range(max_iter):
        for s0 in env.S0:
            search(s0,0)
    return [np.argmax(Q[s]) for s in env.S]

    
pi = uct(max_iter=100, max_d=100, beta=.9)


In [3]:
# with open("mcts.txt", "a") as f:
#     for item in pi:
#         f.write(f'{item}\n')


In [6]:
env = Texas_Holdem_Simp()
s =  np.random.choice(env.S0)
env.plot(s)

----(1)-----
|P2(3): K♥ |
|(2): K♠-**|
|P1(5): Q♥ |
------------


In [9]:
a = pi[s]
print(a)
env.plot(np.random.choice([s_ for s_ in env.S if env.T(s,a,s_)>0]))

1
----(2)-----
|P2(2): K♥ |
|(4): K♠-J♠|
|P1(4): Q♥ |
------------
